In [20]:
# Guide to download emoji in

# import demoji
# demoji.download_codes()

... OK (Got response in 1.52 seconds)
Writing emoji data to C:\Users\rache\.demoji/codes.json ...
... OK


### Import Libraries and Package

In [1]:
# importing libraries
import pandas as pd
import numpy as np
import demoji
import sys
import re as re
import nltk
import string, unicodedata
from nltk.stem.porter import *

from nltk.corpus import stopwords

from datetime import datetime, timedelta

### Upload the Data File

In [11]:
data_file = "all_data_raw.csv"
data = pd.read_csv(data_file)
data.columns = data.columns.str.strip().str.replace(" ","_") # reason for doing this is cause cannot get columns with space into a list

#lower case
data['Review'] = [word.lower() for word in data['Review']]

data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12506 entries, 0 to 12505
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed:_0        12506 non-null  int64  
 1   Platform          12506 non-null  object 
 2   Brand             12506 non-null  object 
 3   Category          12506 non-null  object 
 4   Product_Name      12506 non-null  object 
 5   Price             12506 non-null  object 
 6   Reviewer          12462 non-null  object 
 7   Review            12506 non-null  object 
 8   Product_Purchase  8593 non-null   object 
 9   Ratings           12462 non-null  float64
 10  Date_Of_Review    12462 non-null  object 
 11  Response          12462 non-null  object 
dtypes: float64(1), int64(1), object(10)
memory usage: 1.1+ MB


,Unnamed:_0,Platform,Brand,Category,Product_Name,Price,Reviewer,Review,Product_Purchase,Ratings,Date_Of_Review,Response
0,0,Lazada,Garnier,Makeup Removers,Micellar Cleansing Water All-in-1 Biphase (For...,$13.59,Jhudelin R.,recieved the items timely however when i open ...,Volume (ml):400,3.0,13-Feb-20,no
1,1,Lazada,Garnier,Makeup Removers,Micellar Cleansing Water All-in-1 Biphase (For...,$13.59,Jasmine G.,good buy at a bundle deal! one of my favourite...,Volume (ml):400,5.0,29-Dec-18,yes
2,2,Lazada,Garnier,Makeup Removers,Micellar Cleansing Water All-in-1 Biphase (For...,$13.59,Lazada Customer,purchased 3 bottles during the festival. have ...,Volume (ml):400,5.0,19-Jul-19,no
3,3,Lazada,Garnier,Makeup Removers,Micellar Cleansing Water All-in-1 Biphase (For...,$13.59,Lazada Customer,fast delivery. hassle free.,Volume (ml):400,5.0,11-Sep-19,no
4,4,Lazada,Garnier,Makeup Removers,Micellar Cleansing Water All-in-1 Biphase (For...,$13.59,wendy A.,well received. delivery is consider fast too a...,Volume (ml):400,5.0,17-Dec-19,no


### Transformation and Initalise of data

In [3]:
# make the old csv data into a list
id_list = data['Unnamed:_0'].tolist()
platform_list = data['Platform'].tolist()
brand_list = data['Brand'].tolist()
category_list = data['Category'].tolist()
product_name_list = data['Product_Name'].tolist()
price_list = data['Price'].tolist()
reviewer_list = data['Reviewer'].tolist()
review_list = data['Review'].tolist()
product_variation_list = data['Product_Purchase'].tolist()
rating_list = data['Ratings'].tolist()
date_review_list = data["Date_Of_Review"].tolist()
response_list = data["Response"].tolist()

# initialising the new columns
id_csv=[]
platform_csv=[]
brand_csv = []
category_csv = []
product_name_csv = []
prices_csv = []
reviewer_csv =[]
review_splitted_csv = []
review_csv=[]
product_variation_csv = []
rating_csv = []
date_review_csv = []
response_csv = []
emoji_csv =[]

# this is the length of the csv old data
print ("The length of the dataset is:", len(brand_list))

The length of the dataset is: 12506


### Input the Functions

In [ ]:
# spacy for lemmatization
import spacy

def lemmatization(texts):
    """https://spacy.io/api/annotation"""
    doc = sp(texts)
    
    texts_out = ([token.lemma_ for token in doc])
    return texts_out

In [4]:
#upload the appos
from word_dict import appos
# print(appos)

In [5]:
def reduce_lengthening(text):
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text)

In [ ]:
# New stopWord list
from nltk.corpus import stopwords

to_remove = ['not', 'no','but','if']
new_stopwords = set(stopwords.words('english')).difference(to_remove)
# print(new_stopwords)

### Transform the Date Format

In [1]:
new_date =[]
for i in range(len(brand_list)):
    
    date_review = date_review_list[i]
    
    scrape_date = datetime(2020, 3, 23)
    
    if platform_list[i] == 'Lazada' and not pd.isna(date_review) :
        if 'days ago' in date_review or 'day ago' in date_review :
            day = date_review.replace('days ago', '').replace('day ago', '')
            date_review = scrape_date - timedelta(days=int(day))
            date_review = date_review.strftime("%d-%m-%y")
#             print("trans",date_review)

        if 'weeks ago' in date_review or 'week ago' in date_review:
            day = date_review.replace('weeks ago', '').replace('week ago', '')
            day = int(day)*7
            date_review = scrape_date - timedelta(days=day)
            date_review = date_review.strftime("%d-%m-%y")
#             print("trans",date_review)

        if 'hours ago' in date_review or 'hour ago' in date_review:
            hours = date_review.replace('hours ago', '').replace('hour ago', '')
            date_review = scrape_date - timedelta(hours=int(hours))
            date_review = date_review.strftime("%d-%m-%y")
    
#       print("trans",date_review)
        new_date.append(date_review)
    else:
#         print(platform_list[i])
        new_date.append(date_review)
        
data["Date_Of_Review"] = new_date

new_date_review_list = data["Date_Of_Review"].tolist()

NameError: name 'brand_list' is not defined

### Preprocess the Sentences

In [7]:
import itertools
from spellchecker import SpellChecker
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize


spell = SpellChecker()

for i in range(len(brand_list)):
    # current_review is a string
    current_review = review_list[i]
    current_review = current_review.strip().replace("’","'")
    
    sentences = sent_tokenize(current_review)
    
    #print(processed_review)
    for j in range(len(sentences)):
        emoji_extract=""
        processed_review = sentences[j]
        
        # extract emoji from each sentence
        emoji_dictionary = demoji.findall(processed_review)
        if len(emoji_dictionary) == 0:
            emoji_extract=""
        else:
            temp_list = ""
            for key in emoji_dictionary:
                temp_list += key + " "
            temp_list = temp_list[0:-1]
            emoji_extract= temp_list
        
        # Negation handling
        processed_review = processed_review.split()
        processed_review =[appos[w] if w in appos else w for w in processed_review]
        processed_review = " ".join(processed_review) 

        processed_review = processed_review.strip()
        
        # Remove all the special characters
        processed_review = re.sub(r'\W', ' ', processed_review)
        # Removing prefixed 'b'
        processed_review = re.sub(r'^b\s+', '', processed_review)
        # remove all single characters contains a white space character
        processed_review= re.sub(r'^[a-zA-Z]$', ' ', processed_review)
        # Substituting multiple spaces with single space
        processed_review = re.sub(r'\s+', ' ', processed_review, flags=re.I)
        processed = processed_review.strip()

        
        # final check if the processed_review is empty
        final = []
        if processed != "":
            tokenized = word_tokenize(processed)
            #removed additional letters eg "gooooooodddddd" to "good"
            for w in tokenized:
                processed_2 = reduce_lengthening(w)
                processed_2 = spell.correction(processed_2)
                final.append(processed_2)
            final =' '.join(final)
            
            # append into a dictionary
            id_csv.append(id_list[i])
            platform_csv.append(platform_list[i])
            brand_csv.append(brand_list[i])
            category_csv.append(category_list[i])
            product_name_csv.append(product_name_list[i])
            prices_csv.append(price_list[i])
            reviewer_csv.append(reviewer_list[i])
            review_csv.append(review_list[i])
            review_splitted_csv.append(final)
            product_variation_csv.append(product_variation_list[i])
            rating_csv.append(rating_list[i])
            date_review_csv.append(new_date_review_list[i])
            response_csv.append(response_list[i])
            emoji_csv.append(emoji_extract)
            
#Store into a new data frame
new_data = {'ID':id_csv,'Platform':platform_csv ,'Brand':brand_csv, 'Category': category_csv, 'Product_Name ': product_name_csv, 'Price':prices_csv ,'Reviewer':reviewer_csv,'Review':review_csv,'Product_Purchase':product_variation_csv,'Ratings':rating_csv,'Date_Of_Review':date_review_csv,'Response': response_csv,'Review_splitted':review_splitted_csv,'Emoji':emoji_csv}
new_data = pd.DataFrame.from_dict(new_data)
new_data['Date_Of_Review'] = pd.to_datetime(new_data['Date_Of_Review'])
new_data.head()
new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21354 entries, 0 to 21353
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ID                21354 non-null  int64         
 1   Platform          21354 non-null  object        
 2   Brand             21354 non-null  object        
 3   Category          21354 non-null  object        
 4   Product Name      21354 non-null  object        
 5   Price             21354 non-null  object        
 6   Reviewer          21310 non-null  object        
 7   Review            21354 non-null  object        
 8   Product Purchase  15363 non-null  object        
 9   Ratings           21310 non-null  float64       
 10  Date Of Review    21310 non-null  datetime64[ns]
 11  Response          21310 non-null  object        
 12  Review_splitted   21354 non-null  object        
 13  Emoji             21354 non-null  object        
dtypes: datetime64[ns](1), 

### Insert the Emoji Ranking File

In [9]:
emoji_sentiment_ranking_file = "Emoji_Sentiment_Data_v1.0.csv"
emoji_sentiment_ranking = pd.read_csv(emoji_sentiment_ranking_file)
emoji_sentiment_ranking.columns = emoji_sentiment_ranking.columns.str.strip().str.replace(" ","_")

### Emoji Sentiment

In [13]:
emoji_unicode = emoji_sentiment_ranking['Unicode_codepoint'].tolist()
occurences = emoji_sentiment_ranking['Occurrences'].tolist()
positive = emoji_sentiment_ranking['Positive'].tolist()
neutral = emoji_sentiment_ranking['Neutral'].tolist()
negative = emoji_sentiment_ranking['Negative'].tolist()

def getEmojiSentiment(emoji_string):
    positive_sentiment = 0
    neutral_sentiment = 0
    negative_sentiment = 0
    emoji_splitted = emoji_string.split(" ")
    
    for i in range(len(emoji_splitted)):
        current_emoji = emoji_splitted[i]
        current_emoji = current_emoji[0]
        if len(current_emoji) == 1:
            unicode = '0x{:X}'.format(ord(current_emoji)).lower()
            
            if unicode in emoji_unicode:
                find_unicode_index = emoji_unicode.index(unicode)
                
                if int(positive[find_unicode_index]) >= int(neutral[find_unicode_index]) and int(positive[find_unicode_index]) >= int(negative[find_unicode_index]):
                        positive_sentiment += 1
                elif int(negative[find_unicode_index]) >= int(neutral[find_unicode_index]) and int(negative[find_unicode_index]) >= int(positive[find_unicode_index]):
                        negative_sentiment += 1
                elif int(neutral[find_unicode_index]) >= int(negative[find_unicode_index]) and int(neutral[find_unicode_index]) >= int(positive[find_unicode_index]):
                        neutral_sentiment += 1
        else: 
            pass

    if positive_sentiment > neutral_sentiment and positive_sentiment > negative_sentiment:
        return ("1")
    elif negative_sentiment > neutral_sentiment and negative_sentiment > positive_sentiment:
        return ("-1")
    else:
        return ("0")

In [11]:
emoji_polarity =[]
emoji = new_data['Emoji'].tolist()

for each_emoji in emoji:
    current_review = each_emoji
    print (current_review)
    if current_review != "":
        emoji_polarity.append(getEmojiSentiment(current_review))
        print(getEmojiSentiment(current_review))
    else:
        emoji_polarity.append("")
    print ("\n")

new_data['Emoji_polarity'] = emoji_polarity

### Data Preprocessing

In [ ]:
tokenized_review = new_data["Review_splitted"]

review_cleaned=[]

stemmer = PorterStemmer()
sp = spacy.load('en', disable=['parser', 'ner'])

for sentence in tokenized_review:
    sentence = sentence.strip()
    review = lemmatization(sentence)
    review_stopremoved = [w for w in review if w not in new_stopwords]
    review_cleaned.append(review_stopremoved)
    
new_data['tokenized_review'] = review_cleaned  
new_data.head()

### Import the Positive , Negative , Increment , Decrement and Inverse Words

In [ ]:
import nltk 

pos_lexicon = 'dict/positive-words.txt'
neg_lexicon = 'dict/negative-words.txt'
inc_words = 'dict/increment-words.txt'
dec_words = 'dict/decrement-words.txt'
inv_words = 'dict/inverse-words.txt'


# Read the positive sentiment lexicon.
pos_dict = {}
f = open(pos_lexicon, 'r', encoding = "ISO-8859-1")
for line in f:
    line = line.strip()
    pos_dict[line] = 1
f.close()

# Read the negative sentiment lexicon.
neg_dict = {}
f = open(neg_lexicon, 'r', encoding = "ISO-8859-1")
for line in f:
    line = line.strip()
    neg_dict[line] = 1
f.close()


# Read the increment words.
inc_dict = {}
f = open(inc_words, 'r', encoding = "ISO-8859-1")
for line in f:
    line = line.strip()
    inc_dict[line] = 1
f.close()

# Read the decrement words.
dec_dict = {}
f = open(dec_words, 'r', encoding = "ISO-8859-1")
for line in f:
    line = line.strip()
    dec_dict[line] = 1
f.close()

# Read the inverse words.
inv_dict = {}
f = open(inv_words, 'r', encoding = "ISO-8859-1")
for line in f:
    line = line.strip()
    inv_dict[line] = 1
f.close()


### Lexicon-based Sentiment Polarity Classification

In [ ]:
predicted_labels = []
score_label=[]
normalise_score_label=[]

for sent in review_cleaned:
    total_score = 0
    print(sent)
    for w in sent:
        score = 0
        if w in pos_dict:
            score = 1
        elif w in neg_dict:
            score = -1
        #check previous word
        position = sent.index(w)
        bigram_word = sent[(position-1):position+1]
        print("bigram",bigram_word)
#         trigram_word = sent[(position-3):position]
#         print("tigram",trigram_word)
        if len(bigram_word) > 0:
            print("previous",bigram_word[0])
            if bigram_word[0]in inc_dict: 
                score *= 2.0
            elif bigram_word[0]in dec_dict:
                score /=2
            elif bigram_word[0]in inv_dict:
                score *= -1  
        total_score+=score
    if len(sent)> 0:
        normalised_score  = total_score/(len(sent))
    else:
        normalised_score = 0
    normalise_score_label.append(normalised_score)
    
    if normalised_score >0.1:
        predicted_labels.append('1')
        
    elif normalised_score <-0.1:
        predicted_labels.append('-1')
        
    else:
        predicted_labels.append('0')
        
    score_label.append(int(total_score))
    
new_data["Polarity"] = predicted_labels

### Combine Polarity for Splitted Review

In [ ]:
overall_polarity =[]

polarity = new_data['Polarity'].tolist()
emoji_polarity = new_data['Emoji_polarity'].tolist()

for i in range(len(polarity)):
    sentiment_pol = int(polarity[i])
    emoji_pol = int(emoji_polarity[i])
    if emoji_pol == -1 and sentiment_pol > -1:
        overall_polarity.append("-1")
    else:
        overall_polarity.append(sentiment_pol)
new_data["Overall_polarity"] = overall_polarity
new_data.to_csv('Data/all_data_clean_split.csv')